# 0.0 Imports and Help Functions

In [20]:
# Imports
import pandas as pd
import numpy as np

# Demographics
import geopandas as gpd
import brazilcep

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display import HTML
from IPython.display import Image

# Web scraping
import requests
from bs4 import BeautifulSoup

In [21]:
# Set the option to display all columns in the DataFrame
pd.set_option('display.max_columns', None)

In [22]:
def jupyter_settings():
    %matplotlib inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

jupyter_settings()

# 1. Loading Data

## 1.1 Loading

In [105]:
dados_rt = pd.read_csv('sp.csv', dtype=object)
dados_rt.head()

,AP_MVM,AP_CONDIC,AP_GESTAO,AP_CODUNI,AP_AUTORIZ,AP_CMP,AP_PRIPAL,AP_VL_AP,AP_UFMUN,AP_TPUPS,AP_TIPPRE,AP_MN_IND,AP_CNPJCPF,AP_CNPJMNT,AP_CNSPCN,AP_COIDADE,AP_NUIDADE,AP_SEXO,AP_RACACOR,AP_MUNPCN,AP_UFNACIO,AP_CEPPCN,AP_UFDIF,AP_MNDIF,AP_DTINIC,AP_DTFIM,AP_TPATEN,AP_TPAPAC,AP_MOTSAI,AP_OBITO,AP_ENCERR,AP_PERMAN,AP_ALTA,AP_TRANSF,AP_DTOCOR,AP_CODEMI,AP_CATEND,AP_APACANT,AP_UNISOL,AP_DTSOLIC,AP_DTAUT,AP_CIDCAS,AP_CIDPRI,AP_CIDSEC,AP_ETNIA,AR_SMRD,AR_CID10,AR_LINFIN,AR_ESTADI,AR_GRAHIS,AR_DTIDEN,AR_TRANTE,AR_CIDINI1,AR_DTINI1,AR_CIDINI2,AR_DTINI2,AR_CIDINI3,AR_DTINI3,AR_CONTTR,AR_DTINTR,AR_FINALI,AR_CIDTR1,AR_CIDTR2,AR_CIDTR3,AR_NUMC1,AR_INIAR1,AR_INIAR2,AR_INIAR3,AR_FIMAR1,AR_FIMAR2,AR_FIMAR3,AR_NUMC2,AR_NUMC3,AP_NATJUR
0,200801,EP,350000,2030705,3508205721108,200801,0304010090,744.16,354140,39,20,I,51397248000179,NaN,|{|{}{{{,4,65,M,01,354240,010,19570000,0,1,20080102,20080331,04,1,21,0,0,1,0,0,NaN,E350000016,NaN,0000000000000,0000000,20080102,NaN,0000,C12,0000,NaN,NaN,C12,3,3,G3,20071105,N,NaN,NaN,NaN,NaN,NaN,NaN,S,20080102,1,C12,C770,NaN,072,20080102,20080102,NaN,20080228,20080207,NaN,021,000,NaN
1,200801,EP,350000,2025507,3508205573565,200801,0304010090,749.64,352900,05,61,M,09161265000146,09161265000146,{{|}}}}|,4,51,F,99,352900,010,17514691,0,1,20080123,20080331,04,1,21,0,0,1,0,0,NaN,E350000029,01,0000000000000,2025507,20080123,20080123,0000,C56,0000,NaN,NaN,C56,S,4,00,20071120,NaN,C772,20080115,C774,20080116,NaN,NaN,N,20080115,5,C772,C774,NaN,36,20080123,20080123,NaN,20080331,20080331,NaN,36,NaN,NaN
2,200801,EP,350000,2030705,3508205718842,200801,0304010090,1475.8,354140,39,20,I,51397248000179,NaN,{{||{|~,4,80,M,01,354140,010,19015320,0,1,20080102,20080331,04,1,21,0,0,1,0,0,NaN,E350000016,NaN,0000000000000,0000000,20080102,NaN,0000,C61,0000,NaN,NaN,C61,3,4,G2,20070423,N,NaN,NaN,NaN,NaN,NaN,NaN,S,20080102,5,C61,NaN,NaN,116,20080102,NaN,NaN,20080220,NaN,NaN,000,000,NaN
3,200801,EP,350000,2773171,3508204191613,200801,0304010286,731.64,353060,39,20,I,48095202000190,NaN,{{{{|},4,53,M,01,353060,010,08738120,0,1,20080101,20080331,04,1,21,0,0,1,0,0,NaN,E350000027,NaN,0000000000000,0000000,NaN,NaN,0000,C710,0000,NaN,NaN,C710,3,NaN,GX,20070827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,20071204,1,C710,NaN,NaN,070,20080101,NaN,NaN,20080331,NaN,NaN,000,000,NaN
4,200801,EP,350000,2773171,3508204191712,200801,0304010294,396.0,353060,39,20,I,48095202000190,NaN,{|~|{,4,52,M,01,351880,035,07181140,0,1,20080101,20080331,04,1,21,0,0,1,0,0,NaN,E350000027,NaN,0000000000000,0000000,NaN,NaN,0000,C61,0000,NaN,NaN,C61,3,3,G2,20060104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,20071130,1,C61,NaN,NaN,140,20080101,NaN,NaN,20080331,NaN,NaN,000,000,NaN


## 1.1 Renaming columns

In [106]:
dados_sp_rt1 = dados_rt.copy()

In [107]:
# First we drop the column AP_SMRD
dados_sp_rt1 = dados_sp_rt1.drop(columns=['AR_SMRD'])

In [108]:
# renaming columns
dict_rename = {
    'AP_MVM': 'data_processamento',
    'AP_CONDIC': 'sigla_tipo_gestao',
    'AP_GESTAO': 'codigo_uf_municipio_gestao',
    'AP_CODUNI': 'codigo_cnes',
    'AP_AUTORIZ': 'numero_apac_autorizacao',
    'AP_CMP': 'data_atendimento_competencia',
    'AP_PRIPAL': 'procedimento_principal_apac',
    'AP_VL_AP': 'valor_total_apac_aprovado',
    'AP_UFMUN': 'codigo_uf_municipio_estabelecimento',
    'AP_TPUPS': 'tipo_estabelecimento',
    'AP_TIPPRE': 'tipo_prestador',
    'AP_MN_IND': 'estabelecimento_mantido_individual',
    'AP_CNPJCPF': 'cnpj_estabelecimento_executante',
    'AP_CNPJMNT': 'cnpj_mantenedora',
    'AP_CNSPCN': 'numero_cns_paciente',
    'AP_COIDADE': 'codigo_idade_paciente',
    'AP_NUIDADE': 'idade_paciente',
    'AP_SEXO': 'sexo_paciente',
    'AP_RACACOR': 'raca_cor_paciente',
    'AP_MUNPCN': 'codigo_uf_municipio_residencia_paciente',
    'AP_UFNACIO': 'nacionalidade_paciente',
    'AP_CEPPCN': 'cep_paciente',
    'AP_UFDIF': 'uf_residencia_diferente_uf_estabelecimento',
    'AP_MNDIF': 'municipio_residencia_diferente_municipio_estabelecimento',
    'AP_DTINIC': 'data_inicio_validade',
    'AP_DTFIM': 'data_fim_validade',
    'AP_TPATEN': 'tipo_atendimento_apac',
    'AP_TPAPAC': 'tipo_apac',
    'AP_MOTSAI': 'motivo_saida_permanencia',
    'AP_OBITO': 'indicador_obito',
    'AP_ENCERR': 'indicador_encerramento',
    'AP_PERMAN': 'indicador_permanencia',
    'AP_ALTA': 'indicador_alta',
    'AP_TRANSF': 'indicador_transferencia',
    'AP_DTOCOR': 'data_ocorrencia',
    'AP_CODEMI': 'codigo_orgao_emissor',
    'AP_CATEND': 'carater_atendimento',
    'AP_APACANT': 'numero_apac_anterior',
    'AP_UNISOL': 'codigo_cnes_estabelecimento_solicitante',
    'AP_DTSOLIC': 'data_solicitacao',
    'AP_DTAUT': 'data_autorizacao',
    'AP_CIDCAS': 'cid_causas_associadas',
    'AP_CIDPRI': 'cid_principal',
    'AP_CIDSEC': 'cid_secundario',
    'AP_ETNIA': 'etnia_paciente',
    'AR_SMRD': 'somatoria_multiplos_registros_dados', # Inferred from context, as specific description was missing for AR_SMRD [cite: 145]
    'AR_CID10': 'cid_10_topografia',
    'AR_LINFIN': 'linfonodos_regionais_invadidos',
    'AR_ESTADI': 'estadio_uicc',
    'AR_GRAHIS': 'grau_histopatologico',
    'AR_DTIDEN': 'data_identificacao_patologica',
    'AR_TRANTE': 'tratamentos_anteriores',
    'AR_CIDINI1': 'cid_1_tratamento_anterior',
    'AR_DTINI1': 'data_inicio_1_tratamento_anterior',
    'AR_CIDINI2': 'cid_2_tratamento_anterior',
    'AR_DTINI2': 'data_inicio_2_tratamento_anterior',
    'AR_CIDINI3': 'cid_3_tratamento_anterior',
    'AR_DTINI3': 'data_inicio_3_tratamento_anterior',
    'AR_CONTTR': 'continuidade_tratamento',
    'AR_DTINTR': 'data_inicio_tratamento_solicitado',
    'AR_FINALI': 'finalidade_tratamento',
    'AR_CIDTR1': 'cid_topografico_1',
    'AR_CIDTR2': 'cid_topografico_2',
    'AR_CIDTR3': 'cid_topografico_3',
    'AR_NUMC1': 'numero_campo_insercoes_1',
    'AR_INIAR1': 'data_inicio_1',
    'AR_INIAR2': 'data_inicio_2',
    'AR_INIAR3': 'data_inicio_3',
    'AR_FIMAR1': 'data_fim_1',
    'AR_FIMAR2': 'data_fim_2',
    'AR_FIMAR3': 'data_fim_3',
    'AR_NUMC2': 'numero_campo_insercoes_2',
    'AR_NUMC3': 'numero_campo_insercoes_3',
    'AP_NATJUR': 'codigo_natureza_juridica'
}

dados_sp_rt1 = dados_sp_rt1.rename(columns=dict_rename)

## 1.2 Mapping apac procedures

In [109]:
dados_sp_rt2 = dados_sp_rt1.copy()

In [110]:
# Changing column "procedimento_principal_apac" to object
# dados_sp_rt2["procedimento_principal_apac"] = dados_sp_rt2["procedimento_principal_apac"].astype(str).apply(lambda x: '0' + x[0:9])

In [111]:
# Dictionary with the mapping of APAC procedures
apac_procedures = {'0304010014': 'Betaterapia dérmica', '0304010022': 'Betaterapia oftálmica', '0304010030': 'Betaterapia para profilaxia de pterígio', 
                        '0304010049': 'Braquiterapia', '0304010057': 'Braquiterapia com fios de iridium', '0304010065': 'Braquiterapia com Iodo 125 / Ouro 198', 
                        '0304010073': 'Braquiterapia de alta taxa de dose', '0304010081': 'Verificação por imagem em radioterapia', '0304010090': 'Cobaltoterapia', 
                        '0304010103': 'Implantação de halo para radiocirurgia', '0304010120': 'Irradiação de corpo inteiro pré-transplante de medula óssea', 
                        '0304010138': 'Irradiação de meio corpo', '0304010146': 'Irradiação de pele total', '0304010154': 'Máscara ou imobilização personalizada', 
                        '0304010162': 'Moldagem em colo e/ou corpo do útero', '0304010189': 'Planejamento complexo', '0304010197': 'Planejamento de braquiterapia de alta taxa de dose', 
                        '0304010200': 'Planejamento simples', '0304010219': 'Radiocirurgia', '0304010235': 'Radioterapia de doença ou condição benigna', 
                        '0304010243': 'Radioterapia estereotática fracionada', '0304010260': 'Roentgenterapia', '0304010286': 'Radioterapia com acelerador linear só de fótons', 
                        '0304010294': 'Radioterapia com acelerador linear de fótons e elétrons', '0304010308': 'Colimação personalizada', '0304010316': 'Planejamento para radioterapia conformada tridimensional', 
                        '0304010324': 'Moldagem/Implante em mucosa (por tratamento completo)', '0304010332': 'Moldagem/Implante em pele/mucosa (por tratamento completo)',
                        '0304010367': 'Radioterapia de Cabeça e Pescoço', '0304010367': 'RADIOTERAPIA DE CABEÇA E PESCOÇO', '030401037': 'RADIOTERAPIA DO APARELHO DIGESTIVO', 
                        '0304010367': 'RADIOTERAPIA DE CABEÇA E PESCOÇO', '0304010375': 'RADIOTERAPIA DO APARELHO DIGESTIVO', '0304010383': 'RADIOTERAPIA DE TRAQUEIA, BRÔNQUIO, PULMÃO, PLEURA E MEDIASTINO', 
                        '0304010391': 'RADIOTERAPIA DE OSSOS/CARTILAGENS/PARTES MOLES', '0304010405': 'RADIOTERAPIA DE PELE', '0304010413': 'RADIOTERAPIA DE MAMA', '0304010421': 'RADIOTERAPIA DE CÂNCER GINECOLÓGICO', 
                        '0304010430': 'BRAQUITERAPIA GINECOLÓGICA', '0304010448': 'RADIOTERAPIA DE PÊNIS', '0304010456': 'RADIOTERAPIA DE PRÓSTATA', '0304010464': 'BRAQUITERAPIA DE PRÓSTATA', '0304010472': 'RADIOTERAPIA DO APARELHO URINÁRIO', 
                        '0304010480': 'RADIOTERAPIA DE OLHOS E ANEXOS'}

dados_sp_rt2["procedimento_principal_apac_nome"] = dados_sp_rt2["procedimento_principal_apac"].map(apac_procedures)

## 1.3 Correcting type of data

In [112]:
dados_sp_rt3 = dados_sp_rt2.copy()

In [113]:
# Type -> period
period_list = ['data_processamento', 'data_atendimento_competencia']
for period in period_list:
    dados_sp_rt3[period] = pd.to_datetime(dados_sp_rt3[period], format='%Y%m').astype('period[M]')

In [114]:
# Type -> date
date_list = ['data_inicio_validade', 'data_fim_validade', 'data_solicitacao', 'data_autorizacao', 'data_identificacao_patologica', 
 'data_inicio_1_tratamento_anterior', 'data_inicio_2_tratamento_anterior', 'data_inicio_3_tratamento_anterior', 'data_inicio_tratamento_solicitado',
 'data_inicio_1', 'data_inicio_2', 'data_inicio_3', 'data_fim_1', 'data_fim_2', 'data_fim_3']

for date in date_list:
    dados_sp_rt3[date] = pd.to_datetime(dados_sp_rt3[date], format='%Y%m%d', errors='coerce')

In [115]:
# Type -> float
dados_sp_rt3['valor_total_apac_aprovado'] = dados_sp_rt3['valor_total_apac_aprovado'].astype(float)

## 1.4 Dropping columns and creating column CID10 with just 2 digits

In [116]:
# Dropping columns - remove column 'tipo_prestador' because this info is not used more in datasus
dados_sp_rt4 = dados_sp_rt3.drop(['tipo_prestador', 'numero_apac_autorizacao', 'cnpj_mantenedora', 'numero_cns_paciente', 'cid_causas_associadas', 'numero_campo_insercoes_1', 'numero_campo_insercoes_2', 'numero_campo_insercoes_3'], axis=1).copy()

In [117]:
# Columns CID10
dados_sp_rt4['cid10'] = dados_sp_rt4['cid_principal'].apply(lambda x: x[:3])

## 1.5 Creating column sus_paulista

In [118]:
# Transformig data in a dictionary
sus_paulista = pd.read_csv('.\\arquivos_auxiliares\\tables\\sus_paulista.csv', dtype=object)

sus_paulista['Valor'] = sus_paulista['Valor'].astype(float)

sus_paulista_dict = sus_paulista.set_index('Unnamed: 0')['Valor'].to_dict()

In [119]:
# Creating column sus_paulista
dados_sp_rt4['valor_total_apac_aprovado_sus_paulista'] = np.where(
    dados_sp_rt4['data_processamento'] >= '2023-08',
    dados_sp_rt4['procedimento_principal_apac'].map(sus_paulista_dict), 0)

## 1.6 Creating column city name

In [120]:
# Creating dict to map
municipios_sp = pd.read_excel('.\\arquivos_auxiliares\\tables\\codigosdosmunicipios_sp.xls', dtype=object)

# Excluding last character to match with main data
municipios_sp['cod'] = municipios_sp['Código do Município'].astype(str).apply(lambda x: x[0:6])

municipios_sp_dict = municipios_sp.set_index('cod')['Município'].to_dict()

# Creating column city_cnes -> institution's city
dados_sp_rt4['cidade_cnes'] = dados_sp_rt4['codigo_uf_municipio_estabelecimento'].map(municipios_sp_dict)

# Creating column patient_city
dados_sp_rt4['cidade_patient'] = dados_sp_rt4['codigo_uf_municipio_residencia_paciente'].map(municipios_sp_dict)


## 1.7 Creating column 'ano'

In [121]:
dados_sp_rt4['ano'] = dados_sp_rt4['data_processamento'].dt.year

## 1.8 Changing "raca_cor_paciente" to text

In [122]:
dados_sp_rt5 = dados_sp_rt4.copy()

In [123]:
# Dictionary with string names

cor = {"01": "Branca",
       "02": "Preta",
       "03": "Amarela",
       "04": "Parda",
       "05": "Indígena",
       "99": "Ignorado"}

dados_sp_rt5['raca_cor_paciente'] = dados_sp_rt5['raca_cor_paciente'].map(cor)

## 1.9 Creating column "tipo_estabelecimento" by institution type

In [124]:
estabelecimentos = {'01': 'POSTO DE SAUDE', '02': 'CENTRO DE SAUDE/UNIDADE BASICA', '04': 'POLICLINICA', '05': 'HOSPITAL GERAL', '07': 'HOSPITAL ESPECIALIZADO', '15': 'UNIDADE MISTA',
 '22': 'CONSULTORIO ISOLADO', '32': 'UNIDADE MOVEL FLUVIAL', '36': 'CLINICA/CENTRO DE ESPECIALIDADE', '39': 'UNIDADE DE APOIO DIAGNOSE E TERAPIA (SADT ISOLADO)', '40': 'UNIDADE MOVEL TERRESTRE',
 '42': 'UNIDADE MOVEL DE NIVEL PRE-HOSPITALAR NA AREA DE URGENCIA', '43': 'FARMACIA', '50': 'UNIDADE DE VIGILANCIA EM SAUDE', '60': 'COOPERATIVA OU EMPRESA DE CESSAO DE TRABALHADORES NA SAUDE',
 '62': 'HOSPITAL/DIA - ISOLADO', '68': 'CENTRAL DE GESTAO EM SAUDE', '69': 'CENTRO DE ATENCAO HEMOTERAPIA E OU HEMATOLOGICA', '70': 'CENTRO DE ATENCAO PSICOSSOCIAL', '72': 'UNIDADE DE ATENCAO A SAUDE INDIGENA',
 '73': 'PRONTO ATENDIMENTO', '74': 'POLO ACADEMIA DA SAUDE', '75': 'TELESSAUDE', '76': 'CENTRAL DE REGULACAO MEDICA DAS URGENCIAS', '77': 'SERVICO DE ATENCAO DOMICILIAR ISOLADO(HOME CARE)',
 '79': 'OFICINA ORTOPEDICA', '80': 'LABORATORIO DE SAUDE PUBLICA', '81': 'CENTRAL DE REGULACAO DO ACESSO', '82': 'CENTRAL DE NOTIFICACAO,CAPTACAO E DISTRIB DE ORGAOS ESTADUAL', '83': 'POLO DE PREVENCAO DE DOENCAS E AGRAVOS E PROMOCAO DA SAUDE',
 '84': 'CENTRAL DE ABASTECIMENTO', '85': 'CENTRO DE IMUNIZACAO'}

In [125]:
# Creating column "tipo_estabelecimento" by institution type
dados_sp_rt5['tipo_estabelecimento_nome'] = dados_sp_rt5['tipo_estabelecimento'].map(estabelecimentos)

## 1.10 Changing column 'sigla_tipo_gestao' with names

In [126]:
dados_sp_rt6 = dados_sp_rt5.copy()

In [127]:
# Creatging a dictionary to map the type of management
gestao = {'EP': 'Estado Pleno', 'PG': 'Pacto de Gestão', 'MN': 'Munic Pleno'}

# Changing column 'sigla_tipo_gestao' with names
dados_sp_rt6['sigla_tipo_gestao'] = dados_sp_rt6['sigla_tipo_gestao'].map(gestao)

## 1.11 Creating column 'motivo_saida' according to https://bvsms.saude.gov.br/bvs/saudelegis/gm/2008/prt0024_10_01_2008.html

In [128]:
# Dictionary to map exit/permanence reason codes
motivo_saida = {
    "11": "Alta Curado",
    "12": "Alta Melhorado", 
    "13": "Alta da Puérpera e permanência do recém-nascido",
    "14": "Alta a pedido",
    "15": "Alta com previsão de retorno para acompanhamento do paciente",
    "16": "Alta por Evasão",
    "17": "Alta da Puérpera e recém-nascido",
    "18": "Alta por Outros motivos",
    "21": "Por características próprias da doença",
    "22": "Por Intercorrência",
    "23": "Por impossibilidade sócio-familiar",
    "24": "Por Processo de doação de órgãos, tecidos e células - doador vivo",
    "25": "Por Processo de doação de órgãos, tecidos e células - doador morto",
    "26": "Por mudança de Procedimento",
    "27": "Por reoperação",
    "28": "Outros motivos",
    "31": "Transferido para outro estabelecimento",
    "41": "Com declaração de óbito fornecida pelo médico assistente",
    "42": "Com declaração de Óbito fornecida pelo Instituto Médico Legal - IML",
    "43": "Com declaração de Óbito fornecida pelo Serviço de Verificação de Óbito - SVO",
    "51": "ENCERRAMENTO ADMINISTRATIVO"
}

# Apply the mapping to create a descriptive column
dados_sp_rt6['motivo_saida_permanencia_nome'] = dados_sp_rt6['motivo_saida_permanencia'].map(motivo_saida)